In [1]:
#loading packages
import pyodbc
import csv
import pandas as pd
import sqlalchemy as sa
import psycopg2
import string

In [2]:
#local file location
fileLoc = r'C:\Users\HStandeffer17\Downloads\cfbstats-com-2006-1-5-0\\'

#array of file names within folder
dfArray = ['conference', 'stadium', 'team', 'game', 'game-statistics', 'player', 'player-game-statistics', 'team-game-statistics', 'drive', 'play', 'punt', 'punt-return', 'reception', 'rush', 'kickoff', 'kickoff-return', 'pass']

#for loop that reads in file name based on fileLoc, and the current string item in dfArray (called "name" here)
for name in dfArray:
    #read csv in based on parameters
    df = pd.read_csv(fileLoc + name + '.csv')
    
    #replace these three lines with one line to remove punctuation
    df.columns = df.columns.str.replace(' ', '')
    df.columns = df.columns.str.replace('-', '')
    df.columns = df.columns.str.replace('/', '')
    df.columns = df.columns.str.replace('1st', 'First')
    df.columns = df.columns.str.lower()
    
    if name == 'player':
        df["firstname"] = df['firstname'].str.replace('[^\w\s]','')
        df["lastname"] = df['lastname'].str.replace('[^\w\s]','')
        
    if name == 'pass':
        df['passerplayercode'] = df['passerplayercode'].astype(int)
        df['receiverplayercode'] = df['passerplayercode'].astype(int)
    
    #removing dashes from table names
    name = name.replace('-', '')
    
    #printing out progress and finally converting DataFrame to SQL
    print(name)
    cleanedName = name + '_cleaned.csv'
    df.to_csv(r'C:\Users\HStandeffer17\Desktop\cleaned\\' + cleanedName, index=False)
    
    conn = psycopg2.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=test user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
    cur = conn.cursor()
    fileString = r'C:\Users\HStandeffer17\Desktop\cleaned\\' + name + '_cleaned.csv'
    with open(fileString, 'r') as f:
        # Notice that we don't need the `csv` module.
        next(f)  # Skip the header row.
        cur.copy_from(f, name, sep=',')
    
    conn.commit()

conference


IntegrityError: duplicate key value violates unique constraint "conference_pkey"
DETAIL:  Key (conferencecode)=(821) already exists.
CONTEXT:  COPY conference, line 1


In [ ]:
cleanedName = 'player_cleaned.csv'
  
conn = psycopg2.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=test2 user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()
fileString = r'C:\Users\HStandeffer17\Desktop\cleaned\\' + cleanedName
print(fileString)
with open(fileString, 'r') as f:
    # Notice that we don't need the `csv` module.
    next(f)  # Skip the header row.
    cur.copy_from(f, 'player', sep=',')
    
conn.commit()